In [12]:
import pandas as pd
import numpy as np
import os

import re
import emoji
from konlpy.tag import Okt

from sklearn.metrics import accuracy_score
from scipy.stats import entropy

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch

import requests
import json

from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

In [7]:
os.chdir("C:/Users/ehddl/Desktop/업무/code/sns-categorizer")

In [ ]:
# from dotenv import load_dotenv
# load_dotenv('config/.env')

# token = os.getenv("HUGGINGFACE_TOKEN")

# model_name = "mistralai/Mistral-7B-v0.1"

# tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     token=token,
#     device_map="auto",  # 자동 GPU/CPU 분배
#     load_in_4bit=True,  # 8GB 환경 필수
#     torch_dtype=torch.float16
# )

In [ ]:
# new_data = pd.read_parquet("media_data_sample.parquet")
# new_data_profile = pd.read_parquet("C:/Users/ehddl/Downloads/merged_data.parquet")

# new_data = new_data[['acnt_id', 'media_cn']]
# new_data_profile = new_data_profile[['acnt_id', 'acnt_sub_nm', 'intro_txt']]

# new_data_profile.dropna(inplace=True)
# new_data.dropna(inplace=True)

# new = pd.merge(new_data_profile, new_data, on='acnt_id')

In [ ]:
# def clean_text(text):
#     if not isinstance(text, str):
#         return ''
    
#     text = emoji.replace_emoji(text, replace='')
#     text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
#     text = text.lower()
#     text = re.sub(r'\s+', ' ', text)

#     return text.strip()

# new['acnt_sub_nm_cleaned'] = new['acnt_sub_nm'].apply(clean_text)
# new['intro_txt_cleaned'] = new['intro_txt'].apply(clean_text)
# new['media_cn_cleaned'] = new['media_cn'].apply(clean_text)
# new = new[~new.apply(lambda row: row.astype(str).str.strip().eq('').any(), axis=1)]
# new = new[['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned']]

In [14]:
# --- 0. 하이퍼파라미터 및 모델 설정 ---
# 학습된 BERT 모델 설정
MODEL_NAME = "kykim/bert-kor-base" # 또는 finetune-bert-kykim 등 님이 학습시킨 모델 경로
FINETUNED_BERT_MODEL_PATH = "muli-columns-kykim-bert-kor" # finetune-bert-kykim 혹은 앙상블 모델 등

# 카테고리 라벨 목록 (BERT 학습 시 사용했던 라벨과 동일해야 함)
category_labels = ['IT', '게임', '결혼/연애', '교육', '다이어트/건강보조식품', '만화/애니/툰', '문구/완구', '미술/디자인', '반려동물', '베이비/키즈', '뷰티', '브랜드공식계정',
                   '사진/여행', '셀럽', '스포츠', '시사', '엔터테인먼트', '여행/관광', '유명장소/핫플', '일상', '자동차/모빌리티', '짤/밈', '취미', '패션', '푸드', '홈/리빙'] # 27개 실제 라벨

# --- 1. BERT 모델 및 토크나이저 로드 ---
bert_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModelForSequenceClassification.from_pretrained(
    FINETUNED_BERT_MODEL_PATH,
    num_labels=len(category_labels)
)
bert_model.eval() # 추론 모드로 전환

def tokenize_three_columns(examples):
    combined_texts = [
        f"{acnt} {bert_tokenizer.sep_token} {intro} {bert_tokenizer.sep_token} {txt}"
        for acnt, intro, txt in zip(
            examples["acnt_sub_nm_cleaned"],
            examples["intro_txt_cleaned"],
            examples["media_cn_cleaned"]
        )
    ]
    
    return bert_tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=512 
    )

predict_dataset = Dataset.from_pandas(new)
predict_dataset = predict_dataset.map(tokenize_three_columns, batched=True)
columns_to_remove = ['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned']
predict_dataset = predict_dataset.remove_columns(columns_to_remove)
predict_dataset.set_format(type="torch", columns=['input_ids', 'attention_mask'])

Map: 100%|██████████| 14370/14370 [00:02<00:00, 4973.99 examples/s]


In [15]:
# --- 2. LLM 설정 (2차 정제기/추론기) ---
# LLM 설정
LLM_MODEL_NAME = "mistral:7b" # Ollama에 다운로드된 모델 이름
LLM_CONFIDENCE_THRESHOLD = 0.8 # BERT 예측 확률이 0.8 미만일 경우에만 LLM 호출
TOP_K_PREDICTIONS = 5 # LLM에게 전달할 BERT의 상위 예측 후보 수


llm = ChatOllama(
    model=LLM_MODEL_NAME,
    format='json', # JSON 형식으로 응답받도록 지시 (프롬프트에서도 명시해야 함)
    temperature=0 # 창의성 없이 일관된 답변을 받도록 온도 0으로 설정
)

# LLM에게 전달할 프롬프트 템플릿
# {text}, {category_labels}, {bert_predictions_info} 변수를 동적으로 채워넣을 것입니다.
prompt_template = PromptTemplate.from_template(
    """
    당신은 텍스트 분류 전문가입니다. 다음 SNS 텍스트를 가장 적절한 하나의 카테고리로 분류하세요.
    - 카테고리 목록: {category_labels}
    - 텍스트: "{text}"
    - BERT 1차 예측 결과: {bert_predictions_info}
    
    최종 카테고리는 카테고리 목록에 있는 이름 중 하나여야 하며, 다른 말 없이 카테고리 이름만 JSON 형식으로 출력하세요.
    예시: {{"category": "음식"}}
    """
)

In [ ]:
# --- 3. 데이터 전처리 및 하이브리드 라벨링 파이프라인 ---
def tokenize_and_predict_batch(new_profile_data, new_media_data):
    new_profile_data = new_profile_data[['acnt_id', 'acnt_sub_nm', 'intro_txt']]
    new_media_data = new_media_data[['acnt_id', 'media_cn']]

    new_profile_data.dropna(inplace=True)
    new_media_data.dropna(inplace=True)

    new = pd.merge(new_profile_data, new_media_data, on='acnt_id')

    def clean_text(text):
        if not isinstance(text, str):
            return ''
        
        text = emoji.replace_emoji(text, replace='')
        text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text)

        return text.strip()

    new['acnt_sub_nm_cleaned'] = new['acnt_sub_nm'].apply(clean_text)
    new['intro_txt_cleaned'] = new['intro_txt'].apply(clean_text)
    new['media_cn_cleaned'] = new['media_cn'].apply(clean_text)
    new = new[~new.apply(lambda row: row.astype(str).str.strip().eq('').any(), axis=1)]
    new = new[['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned']]
    predict_df = new.copy()

    def tokenize_three_columns(examples):
        combined_texts = [
            f"{acnt} {bert_tokenizer.sep_token} {intro} {bert_tokenizer.sep_token} {txt}"
            for acnt, intro, txt in zip(
                examples["acnt_sub_nm_cleaned"],
                examples["intro_txt_cleaned"],
                examples["media_cn_cleaned"]
            )
        ]
        
        return bert_tokenizer(
            combined_texts,
            padding="max_length",
            truncation=True,
            max_length=512 
        )

    predict_dataset = Dataset.from_pandas(new)
    predict_dataset = predict_dataset.map(tokenize_three_columns, batched=True)
    columns_to_remove = ['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'media_cn_cleaned']
    predict_dataset = predict_dataset.remove_columns(columns_to_remove)
    predict_dataset.set_format(type="torch", columns=['input_ids', 'attention_mask'])
    
    # 예측용 TrainingArguments 및 Trainer 설정
    prediction_args = TrainingArguments(
        output_dir="./prediction_output",
        per_device_eval_batch_size=16,
        do_train=False,
        do_predict=True,
        report_to="none",
        disable_tqdm=False,
    )
    trainer = Trainer(model=bert_model, args=prediction_args)

    # 예측 수행
    predictions_output = trainer.predict(predict_dataset)
    logits = predictions_output.predictions
    probabilities = torch.softmax(torch.tensor(logits), dim=-1).numpy()
    predicted_class_indices = np.argmax(logits, axis=-1)
    
    # 4. 결과 DataFrame에 추가
    predict_df['bert_probabilities'] = [probs.tolist() for probs in probabilities]
    predict_df['bert_top_label_idx'] = np.argmax(probabilities, axis=-1)
    predict_df['bert_top_label'] = [category_labels[idx] for idx in predict_df['bert_top_label_idx']]
    predict_df['bert_top_prob'] = np.max(probabilities, axis=-1)
    
    return predict_df


In [22]:
def process_batch_with_hybrid(predict_df):
    """
    BERT 예측 결과를 담은 DataFrame을 받아 LLM으로 정제하는 함수.
    """
    predict_df['final_category'] = predict_df['bert_top_label'] # 초기값은 BERT의 예측으로 설정
    
    for index, row in predict_df.iterrows():
        text = row['media_cn']
        max_prob = row['bert_top_prob']
        bert_top_label = row['bert_top_label']
        probs = np.array(row['bert_probabilities'])

        # LLM 호출 조건: BERT 예측이 불확실한 경우
        if max_prob < LLM_CONFIDENCE_THRESHOLD:
            print(f"\n--- 텍스트 {index+1} 처리 중 ---")
            print(f"[BERT 불확실] 최고 확률 {max_prob:.4f} < {LLM_CONFIDENCE_THRESHOLD}. LLM 호출...")

            # 상위 K개 예측 후보 추출
            top_indices = np.argsort(probs)[-TOP_K_PREDICTIONS:][::-1]
            top_probs = probs[top_indices]
            bert_predictions_info = []
            for idx, prob in zip(top_indices, top_probs):
                label = category_labels[idx]
                bert_predictions_info.append(f"{label} ({prob:.4f})")
            
            # LangChain으로 LLM에 요청
            prompt_variables = {
                "text": text,
                "category_labels": ', '.join(category_labels),
                "bert_predictions_info": ', '.join(bert_predictions_info)
            }
            chain = prompt_template | llm
            
            try:
                llm_response = chain.invoke(prompt_variables)
                llm_output_dict = eval(llm_response.content)
                final_category = llm_output_dict.get('category')
                predict_df.loc[index, 'final_category'] = final_category
                print(f"-> LLM 최종 결정: {final_category}")
            except Exception as e:
                print(f"LLM 호출 실패 또는 응답 파싱 오류: {e}")
                print(f"-> BERT의 예측값({bert_top_label})으로 대체합니다.")
                predict_df.loc[index, 'final_category'] = bert_top_label
        else:
            print(f"\n--- 텍스트 {index+1} 처리 중 ---")
            print(f"[BERT 확실] 최고 확률 {max_prob:.4f} >= {LLM_CONFIDENCE_THRESHOLD}. LLM 스킵.")
            print(f"-> BERT 최종 결정: {bert_top_label}")

    return predict_df

In [23]:
bert_predictions_df = tokenize_and_predict_batch(new_data_profile, new_data)

Map: 100%|██████████| 14370/14370 [00:02<00:00, 5218.70 examples/s]


In [25]:
# final_results_df = process_batch_with_hybrid(bert_predictions_df)

##### simple prompt test

In [ ]:
'''
구조화된, 여러 단계의 파이프라인이 필요할 때 랭체인 사용. 주요 목적은 프롬프트 템프릿, llm, 출력 파서 등 여러 구성 요소를 연결하는 것. 단순한 단발성 프롬프트 보다 더 복잡한 작업에 활용
프로젝트가 커질수록 일반적으로 순수 requests 코드보다 가독성이 좋고 관리가 쉬움
'''

from langchain_ollama import ChatOllama

llm = ChatOllama(
    model='mistral:7b',
    format='json',
    temperature=0
)

In [26]:
data = pd.read_excel("new_multi-columns.xlsx", index_col=0)

In [28]:
data.head()

,acnt_sub_nm_cleaned,intro_txt_cleaned,media_cn_cleaned,predicted_label,prob_0,prob_1,prob_2,prob_3,prob_4,prob_5,...,prob_17,prob_18,prob_19,prob_20,prob_21,prob_22,prob_23,prob_24,prob_25,single_label
0,yjmgoslow,yjm,yjm will drop a new album in the hottest summer,16,0.002786,0.007358,0.003483,0.010550,0.001860,0.012073,...,0.015325,0.008815,0.170553,0.002231,0.011930,0.003620,0.006244,0.005798,0.001646,엔터테인먼트
1,yjmgoslow,yjm,yg,19,0.000232,0.000165,0.000288,0.000441,0.000506,0.000460,...,0.000781,0.000318,0.983935,0.000239,0.006725,0.000145,0.000470,0.000469,0.000190,일상
2,yjmgoslow,yjm,저의 근황은더보기,19,0.000172,0.000131,0.000203,0.000350,0.000259,0.000548,...,0.000723,0.000207,0.989646,0.000156,0.003760,0.000099,0.000315,0.000317,0.000164,일상
3,yjmgoslow,yjm,99년식 김윤호 생일 축하해 주셔서 다들 감사해요,19,0.000497,0.000658,0.010724,0.000958,0.002319,0.001174,...,0.001867,0.003472,0.722453,0.001677,0.003726,0.000671,0.002762,0.016437,0.003005,일상
4,yjmgoslow,yjm,yoooooooo,19,0.000247,0.000170,0.000279,0.000497,0.000495,0.000475,...,0.000853,0.000315,0.983531,0.000241,0.006814,0.000154,0.000521,0.000463,0.000194,일상


In [ ]:
ollama_url = "http://localhost:11434/api/generate"

def ollama_predict(prompt):
    data = {
        "model": "mistral",
        "prompt": prompt,
        "stream": False # 스트리밍 없이 한 번에 응답 받기
    }
    response = requests.post(ollama_url, data=json.dumps(data))
    response.raise_for_status()
    return response.json()['response']

# BERT가 '낮은 확신도'를 보인 데이터
acnt_sub_nm_cleaned = data['acnt_sub_nm_cleaned']
intro_txt_cleaned = data['intro_txt_cleaned']
media_cn_cleaned = data['media_cn_cleaned']
single_label_list = data['single_label_list'].unique()
bert_label = data['single_label']

prompt = f"""
당신은 텍스트 분류 전문가입니다. 다음 정보들의 텍스트를 보고 26개 카테고리 중 하나로 분류하세요.
- 카테고리 목록: {single_label_list}
- 프로필 별명 : {acnt_sub_nm_cleaned}
- 프로필 설명 : {intro_txt_cleaned}
- 게시물 텍스트: {media_cn_cleaned}

BERT 모델은 이 게시물을 '{bert_label}'로 분류했습니다. 당신의 최종 의견은 무엇인가요?
"""

llm_label = ollama_predict(prompt)
print(f"LLM의 최종 의견: {llm_label}")